In [1]:
import pandas as pd
from datetime import timedelta

filepath1 = r'/Users/cartersocha/Downloads/uberDataset6.csv'
uberSet = pd.read_csv(filepath1)

In [2]:
uberSet2 = uberSet
uberSet2 = uberSet2.groupby(['TvShow','PostReleaseDay']).sum().reset_index()
uberSet2['moving'] = uberSet2.groupby(['TvShow','PostReleaseDay'])['ScaledDataAdjust-QuantileTransformer'].transform(lambda x: x.rolling(7,1).mean())

### Moving average

In [3]:
uberSet2.head(15)

,TvShow,PostReleaseDay,Unnamed: 0,SocialMediaValue,ScaledDataAdjust-QuantileTransformer,ScaledDataAdjust-MinMax,ScaledDataAdjust-StandardScaler,ScaledDataAdjust-MaxAbsScaler,ScaledDataAdjust-RobustScaler,ScaledDataAdjust-Normalizer,ScaledDataAdjust-PowerTransformer,EpisodeCount,SeasonNumber,Ignore,DailyReleaseCount,moving
0,ABlackLadySketchShow,0,10745,360.64,1.993450,0.000057,-0.208537,0.000013,-0.234234,1.0,-1.660078,30.0,15.0,5.0,5.0,1.993450
1,ABlackLadySketchShow,1,8614,1077.00,2.639429,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,24.0,12.0,4.0,0.0,2.639429
2,ABlackLadySketchShow,2,10790,1801.24,2.677893,0.000057,-0.208537,0.000013,-0.234234,1.0,-1.660078,30.0,15.0,5.0,0.0,2.677893
3,ABlackLadySketchShow,3,10815,834.76,2.339414,0.000082,-0.207856,0.000038,-0.225225,1.0,-1.274430,30.0,15.0,5.0,0.0,2.339414
4,ABlackLadySketchShow,4,10840,458.64,2.075665,0.000082,-0.207856,0.000038,-0.225225,1.0,-1.274430,30.0,15.0,5.0,0.0,2.075665
5,ABlackLadySketchShow,5,10865,278.16,1.884641,0.000057,-0.208537,0.000013,-0.234234,1.0,-1.660078,30.0,15.0,5.0,0.0,1.884641
6,ABlackLadySketchShow,6,8710,239.44,1.216671,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,24.0,12.0,4.0,0.0,1.216671
7,ABlackLadySketchShow,7,10910,180.25,1.649437,0.000057,-0.208537,0.000013,-0.234234,1.0,-1.660078,30.0,15.0,5.0,5.0,1.649437
8,ABlackLadySketchShow,8,8746,355.24,1.808895,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,24.0,12.0,4.0,0.0,1.808895
9,ABlackLadySketchShow,9,10955,919.76,2.280650,0.000057,-0.208537,0.000013,-0.234234,1.0,-1.660078,30.0,15.0,5.0,0.0,2.280650


In [4]:
firstWeek = uberSet[uberSet['PostReleaseDay'] < 7]
firstWeek = firstWeek[firstWeek['Ignore'] == 1]
firstWeek['ScaledDataAdjust-QuantileTransformer'] = firstWeek['ScaledDataAdjust-QuantileTransformer'].replace(0,.1)
firstWeek['SocialMediaValue'] = firstWeek['SocialMediaValue'].replace(0,.1)
firstWeekSum = firstWeek.groupby(['TvShow','ReleaseCadence'], as_index=False).sum()
firstWeekSum = firstWeekSum[['TvShow','ReleaseCadence','SocialMediaValue','ScaledDataAdjust-QuantileTransformer']]

In [5]:
totalDf = uberSet[uberSet['Ignore'] == 1]
totalDf['SocialMediaValue'] = totalDf['SocialMediaValue'].replace(0,.1)
totalDf = totalDf.groupby(['TvShow','ReleaseCadence'], as_index=False).sum()
totalDf = totalDf[['TvShow','ReleaseCadence','SocialMediaValue','ScaledDataAdjust-QuantileTransformer']]

In [6]:
firstWeekSum.head()

,TvShow,ReleaseCadence,SocialMediaValue,ScaledDataAdjust-QuantileTransformer
0,ABlackLadySketchShow,Weekly,5049.88,14.827163
1,AboutLastNighthbo,Binge,0.70,0.700000
2,Archive81,Binge,42649.28,16.023360
3,Atlantafx,Hybrid,402624.73,26.995722
4,Barry,Weekly,212344.21,28.224639


In [7]:
totalDf.sort_values(['ScaledDataAdjust-QuantileTransformer']).head()

,TvShow,ReleaseCadence,SocialMediaValue,ScaledDataAdjust-QuantileTransformer
1,AboutLastNighthbo,Binge,9.80,0.148148
53,tehrantv,Hybrid,8116.87,59.833905
51,WokeHulu,Binge,6517.23,69.596446
15,HypeHouse,Binge,66522.44,73.989323
17,JoeVsCarole,Binge,10007.52,74.782577


In [8]:
merged = pd.merge(totalDf,firstWeekSum, how='outer', on='TvShow')

In [9]:
merged['multiplier_scaled'] = merged['ScaledDataAdjust-QuantileTransformer_x'] / merged['ScaledDataAdjust-QuantileTransformer_y']

In [10]:
merged['multiplier_unscaled'] = merged['SocialMediaValue_x'] / merged['SocialMediaValue_y']

In [16]:
merged[['TvShow','ReleaseCadence_x','multiplier_scaled','multiplier_unscaled']].sort_values(['multiplier_scaled'],ascending=0).head(20)

,TvShow,ReleaseCadence_x,multiplier_scaled,multiplier_unscaled
10,GildedAgeHBO,Weekly,29.142985,18.845767
31,Peacemaker,Hybrid,19.786642,5.893327
45,TheLegendOfVoxMachina,Hybrid,16.004594,2.828909
35,Severance,Hybrid,14.793396,12.352559
18,KillingIt,Binge,14.471493,21.090290
12,HIMYF,Hybrid,14.309552,7.108112
32,RaisedByWolvesMax,Hybrid,13.434028,5.724212
4,Barry,Weekly,12.266398,10.323921
39,Suspicion,Hybrid,12.106711,17.086628
28,Outlander,Weekly,11.916752,5.075212


In [12]:
analysisDf = merged.groupby(['ReleaseCadence_x'], as_index=False).mean()

In [13]:
analysisDf[['ReleaseCadence_x','multiplier_scaled','multiplier_unscaled']]

,ReleaseCadence_x,multiplier_scaled,multiplier_unscaled
0,Binge,8.172213,5.136646
1,Hybrid,10.778315,7.047054
2,Weekly,11.897911,7.415008
